In [1]:
import warnings
warnings.simplefilter("ignore")
from mpl_toolkits.axes_grid1 import ImageGrid
import matplotlib.pyplot as plt
import pandas as pd
import random
import shutil
import umap
import numpy as np
import os

# Own libraries
import sys
main_path = '/mnt/cephfs/sharedscratch/users/fshahi/Projects/Histomorphological-Phenotype-Learning'
sys.path.append(main_path)
from data_manipulation.data import Data
from data_manipulation.utils import store_data
from models.evaluation.folds import load_existing_split
from models.clustering.data_processing import *

2024-05-01 16:08:05.859363: I tensorflow/stream_executor/platform/default/dso_loader.cc:50] Successfully opened dynamic library libcudart.so.12


In [2]:
dataset = 'Meso'
refrence_dataset = 'Meso_400_subsampled'
fold = 4
resolution = 5.0
groupby = 'leiden_{}'.format(resolution)
meta_field = 'removal'
rep_key = 'z_latent'
clusters_to_remove = [4,15,23,26,27,34,48,51,54,56,65,66,68,73,75,78,81,84,85,88,90,91,92]

In [7]:
data = Data(dataset=dataset, marker='he', patch_h=224, patch_w=224, n_channels=3, batch_size=64, project_path=main_path, load=True)
h5_path = '{}/results/BarlowTwins_3/{}/h224_w224_n3_zdim128/hdf5_{}_he_complete.h5'.format(main_path, dataset, dataset)
frame_clustered = pd.read_csv('{}/results/BarlowTwins_3/{}/h224_w224_n3_zdim128/removal/adatas/{}_he_complete_leiden_{}__fold4.csv'.format(main_path, refrence_dataset, dataset, str(resolution).replace('.', 'p')))
frame_clustered

# frame, dims, rest = representations_to_frame(h5_path, meta_field=meta_field, rep_key=rep_key)
# frame

Meso


,hist_subtype,indexes,labels,original_set,patterns,samples,slides,tiles,leiden_5.0
0,train,0,-1.0,train,train,MESO_340_2,MESO_340_2,48_28.jpeg,87
1,train,1,-1.0,train,train,MESO_132_26,MESO_132_26,17_8.jpeg,23
2,train,2,-1.0,train,train,MESO_334_17,MESO_334_17,3_25.jpeg,0
3,train,3,-1.0,train,train,MESO_271_4,MESO_271_4,35_8.jpeg,49
4,train,4,-1.0,train,train,MESO_244_14,MESO_244_14,49_14.jpeg,57
...,...,...,...,...,...,...,...,...,...
3918218,test,470877,0.0,test,test,MESO_148_22,MESO_148_22,7_24.jpeg,27
3918219,test,470878,0.0,test,test,MESO_145_18,MESO_145_18,23_8.jpeg,24
3918220,test,470879,0.0,test,test,MESO_111_1,MESO_111_1,33_26.jpeg,25
3918221,test,470880,0.0,test,test,MESO_145_18,MESO_145_18,35_23.jpeg,89


,Meso_type,Sex,case_Id,desmoplastic_component,hist_subtype,indexes,labels,original_set,os_event_data,os_event_ind,...,recurrence,samples,slides,smoking_history,stage,tiles,time_to_recurrence,type,wcc_score,leiden_5.0
0,1,Male,411,No,train,0,1.0,train,4.0,1,...,0,MESO_411_5,MESO_411_5,Ex-smoker,N1,68_14.jpeg,1.0,Biphasic,Normal,82
1,0,Male,383,Unknown,train,1,1.0,train,30.0,1,...,0,MESO_383_34,MESO_383_34,Never smoked,N1,31_22.jpeg,1.0,Epithelioid,Normal,30
2,0,Female,453,Unknown,train,2,1.0,train,12.0,1,...,0,MESO_453_23,MESO_453_23,Unknown,N1,90_24.jpeg,9.0,Epithelioid,Normal,59
3,0,Male,365,Unknown,train,3,1.0,train,12.0,1,...,1,MESO_365_12,MESO_365_12,Unknown,N0,13_27.jpeg,10.0,Epithelioid,Normal,39
4,0,Male,81,Unknown,train,4,1.0,train,1.0,1,...,0,MESO_81_9,MESO_81_9,Ex-smoker,N0,45_20.jpeg,1.0,Epithelioid,High,61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3103762,0,Male,148,Unknown,test,429779,0.0,test,18.0,1,...,0,MESO_148_22,MESO_148_22,Unknown,N1,7_24.jpeg,0.0,Epithelioid,Normal,27
3103763,0,Male,145,Unknown,test,429780,0.0,test,11.0,1,...,0,MESO_145_18,MESO_145_18,Unknown,N0,23_8.jpeg,6.0,Epithelioid,Normal,24
3103764,0,Female,111,Unknown,test,429781,0.0,test,13.0,1,...,0,MESO_111_1,MESO_111_1,Never smoked,N0,33_26.jpeg,6.0,Epithelioid,Normal,25
3103765,0,Male,145,Unknown,test,429782,0.0,test,11.0,1,...,0,MESO_145_18,MESO_145_18,Unknown,N0,35_23.jpeg,6.0,Epithelioid,Normal,89


In [9]:
pkl_path = '{}/files/indexes_to_remove/{}/{}.pkl'.format(main_path, refrence_dataset, dataset)
indecies = frame_clustered[frame_clustered[groupby].isin(clusters_to_remove)].index.astype(int).values.tolist()
store_data(indecies, pkl_path)


### Tiled Clusters Images

In [ ]:
def cluster_set_images(frame, data_dicts, cluster_id, groupby, batches=1):
    indexes       = frame[frame[groupby]==cluster_id]['indexes'].values.tolist()
    original_sets = frame[frame[groupby]==cluster_id]['original_set'].values.tolist()
   


    combined      = list(zip(indexes, original_sets))
    random.shuffle(combined)
    combined_plot = sorted(combined[:100*batches])
    combined_plot = combined[:100*batches]

    for batch in range(batches):
        images_cluster = list()
        for index, original_set in combined_plot[100*batch:100*(batch+1)]:
            images_cluster.append(data_dicts[original_set][int(index)]/255.)

        sns.set_theme(style='white')
        fig = plt.figure(figsize=(30, 6))
        fig.suptitle('Cluster %s' % (cluster_id), fontsize=18, fontweight='bold')
        grid = ImageGrid(fig, 111, nrows_ncols=(5, 20), axes_pad=0.1,)

        for ax, im in zip(grid, images_cluster):
            ax.imshow(im)
            ax.set_xticks([])
            ax.set_yticks([])

        plt.show()
        sns.set_theme(style='darkgrid')
        # if you want to see all the images in the cluster, comment the 'break' line
        break


for cluster_id in clusters_to_remove:
    data_dicts = dict()
    data_dicts['train'] = data.training.images
    data_dicts['valid'] = data.validation.images
    data_dicts['test']  = data.test.images
    cluster_set_images(frame_clustered, data_dicts, cluster_id, groupby, batches=100)